# Ingestion del archivo movie_genre.json

In [0]:
dbutils.widgets.text("p_environment", "")
v_env = dbutils.widgets.get("p_environment")
v_env

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
movie_genre_schema = StructType(fields=[
    StructField("movieId", IntegerType(), False),
    StructField("genreId", IntegerType(), False),
])

In [0]:
movie_genre_df = spark.read.json(f'{bronze_folder_path}/movie_genre.json', schema = movie_genre_schema)

In [0]:
type(movie_genre_df)

In [0]:
movie_genre_df.printSchema()

In [0]:
display(movie_genre_df)

### Paso 2 - Renombrar columnas y adicionar columnas "ingestion_date" y "env"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
movies_genres_renamed_df = (movie_genre_df
    .withColumnRenamed("genreId", "genre_id")
    .withColumnRenamed("movieId", "movie_id")
    .transform(add_ingestion_date)
    .withColumn('env', lit(v_env))
)

### Paso 3 - Particionar por movie_id y Escribir datos en el DataLake en formato parquet

In [0]:
# movies_genres_renamed_df.write.mode('overwrite').partitionBy('movie_id').parquet(f'{silver_folder_path}/movie_genres')
movies_genres_renamed_df.write.mode('overwrite').format('parquet').saveAsTable('movie_silver.movies_genres')

In [0]:
%python
display(dbutils.fs.ls("/mnt/moviehistory/silver/movies_genres"))

In [0]:
df = spark.read.parquet('/mnt/moviehistory/silver/movies_genres')
display(df)

In [0]:
dbutils.notebook.exit("Sucess")